## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

In [ ]:
# Import my library:

import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import dataset_comp
import curve_fit

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(dataset_comp)
importlib.reload(curve_fit)

# Maximum Rotational Velocities

## Motivation

Just from the reduction of power on small scales in the initial conditions, one would expect that (at least for the isolated galaxies) the rotation curve peak – for any galaxy of given mass (as measured by $v_\mathrm{max}$) – is at higher radius in the curvaton simulations. This should be especially true for low-mass galaxies (and, of course, this effect is expected to become insignificant towards the higher masses). Indeed, this is observed in the dwarf galaxy rotation curves. To make a clear case for the existence of this systematic effect, I plot $v_\mathrm{max}$ vs. $r_\mathrm{max}$ (the peak radius). 

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
snap_id = 127
sim_ids = ['V1_MR_fix', 'V1_MR_curvaton_p082_fix']
names = ['plain-LCDM', 'spec-p082']
colors = [['black', 'gray'], ['red', 'pink']]
marker = ['o', '^']

m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = 'by_r'
maxdi = 2000 # Maximum distance from LG centre for isolated

Set lower limit for the value of $v_\mathrm{max}$ of the galaxies to be included (separately for satellites and isolated galaxies):

In [ ]:
sat_low = 10
isol_low = 15

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
data = {}
for name, sim_id, m31_ns, mw_ns, col, mark in \
    zip(names, sim_ids, m31, mw, colors, marker):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns,
                  'PlotStyle': {
                      'Color': col,
                      'Marker': mark
                  }
                 }

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data["snapshot"]
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    # Split into satellites:
    if distinction == "by_r":
        masks_sat, mask_isol = dataset_comp.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "by_gn":
        masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
        
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    
    # Prune low-mass:
    mask_sat_low = dataset_comp.prune_vmax(snap, low_lim=sat_low)
    mask_isol_low = dataset_comp.prune_vmax(snap, low_lim=isol_low)
    mask_m31 = np.logical_and(masks_sat[0], mask_sat_low)
    mask_mw = np.logical_and(masks_sat[1], mask_sat_low)
    mask_sat = np.logical_and(np.logical_or.reduce(masks_sat), mask_sat_low)
    mask_isol = np.logical_and(mask_isol, mask_isol_low)

    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Satellite': mask_sat,
        'Isolated': mask_isol,
        'Luminous': mask_lum,
        'Dark': mask_dark
    }
        
    # Add datasets to dictionary:
    data[name]['Vmax'] = vmax
    data[name]['Rmax'] = rmax

---

## Plot

Plot satellites and isolated galaxies on different subplots and add median fits for each dataset.

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(ncols=2, sharey='row')
plt.subplots_adjust(wspace=0.05)

ms = 8 # Marker size
msl = 10
a = 0.9 # Transparency


# Set axis:
for ax in axes:
    ax.set_xscale('log')
    ax.set_box_aspect(0.9) # Set subfigure box side aspect ratio
    
axes[0].set_xlim(10, 110)    
axes[1].set_xlim(15, 110)
axes[0].set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$' )
axes[1].set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$' )

axes[0].set_yscale('log')
axes[0].set_ylim(0.5, 50)    
axes[0].set_ylabel('$r_{\mathrm{max}}[\mathrm{kpc}]$' )

axes[0].set_title('Satellite Galaxies' )
axes[1].set_title('Isolated Galaxies' )

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot dark:
    mask = np.logical_and(entry['Selections']['Satellite'],
                          entry['Selections']['Dark'])
    axes[0].scatter(entry['Vmax'][mask], entry['Rmax'][mask], 
                    s=ms, edgecolor='none', alpha=a,
                    c=entry['PlotStyle']['Color'][1],
                    label="{} non-SF".format(name))
    
    mask = np.logical_and(entry['Selections']['Isolated'],
                          entry['Selections']['Dark'])
    axes[1].scatter(entry['Vmax'][mask], entry['Rmax'][mask], 
                    s=ms, edgecolor='none', alpha=a,
                    c=entry['PlotStyle']['Color'][1],
                    label="{} non-SF".format(name))
    
    # Plot luminous:
    mask = np.logical_and(entry['Selections']['Satellite'],
                          entry['Selections']['Luminous'])
    axes[0].scatter(entry['Vmax'][mask], entry['Rmax'][mask], 
                    s=msl, facecolors='none', alpha=a,
                    edgecolors=entry['PlotStyle']['Color'][1],
                    label="{} SF".format(name))
    
    mask = np.logical_and(entry['Selections']['Isolated'],
                          entry['Selections']['Luminous'])
    axes[1].scatter(entry['Vmax'][mask], entry['Rmax'][mask], 
                    s=msl, facecolors='none', alpha=a,
                    edgecolors=entry['PlotStyle']['Color'][1],
                    label="{} SF".format(name))
        
axes[1].legend(loc='lower right')

### Add Median Curves

In [ ]:
n_median_points = 5
min_dx = 0.05
for i, (name, entry) in enumerate(data.items()):
    
    # For satellites:
    mask = entry['Selections']['Satellite']
    x = np.log10(entry['Vmax'][mask])
    y = np.log10(entry['Rmax'][mask])
    median = curve_fit.median_once_more(
        x, y, n_points_per_bar=n_median_points, min_dx=min_dx
    )
    if median is not None:
        axes[0].plot(10**median[0], 10**median[1], 
                     entry['PlotStyle']['Color'][0], linestyle='--')
    else:
        print("Could not fit median for:", name)
    
    # For isolated:
    mask = entry['Selections']['Isolated']
    x = np.log10(entry['Vmax'][mask])
    y = np.log10(entry['Rmax'][mask])
    median = curve_fit.median_once_more(
        x, y, n_points_per_bar=n_median_points, min_dx=min_dx
    )
    if median is not None:
        axes[1].plot(10**median[0], 10**median[1], 
                     entry['PlotStyle']['Color'][0], linestyle='--')
    else:
        print("Could not fit median for:", name)
        
fig

### Save the Figures

In [ ]:
filename = 'rmax_vs_vmax'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')